In [4]:
rm(list = ls())
library(tidyverse)
brf <- read_csv("BRFSS2015_650.csv")

Rows: 441456 Columns: 37
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (37): NUMWOMEN, GENHLTH, PHYSHLTH, MENTHLTH, POORHLTH, PERSDOC2, ASTHMA3...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
Q1 <- brf %>%
  filter(GENHLTH == 1) %>%
  count() %>%
  pull(n) %>%
  as.integer()

Q1

[1] 76032

In [6]:
Q2 <- brf %>%
  filter(CVDSTRK3 == 1) %>%
  summarise(max_num_women = max(NUMWOMEN, na.rm = TRUE)) %>%
  as.data.frame()

Q2

max_num_women
<dbl>
10


In [7]:
Q3 <- brf %>%
  filter(
    CRGVPERS %in% c(1, 2) &
    (MENTHLTH %in% c(0, 88) | (MENTHLTH >= 1 & MENTHLTH <= 30))
  ) %>%
  mutate(MENTHLTH = ifelse(MENTHLTH == 88, 0, MENTHLTH)) %>%
  group_by(CRGVPERS) %>%
  summarise(
    mean_mental = mean(MENTHLTH, na.rm = TRUE),
    sd_mental = sd(MENTHLTH, na.rm = TRUE)
  ) %>%
  round(2) %>%
  as.data.frame()

Q3

CRGVPERS,mean_mental,sd_mental
<dbl>,<dbl>,<dbl>
1,5.23,9.32
2,4.14,8.40


In [8]:
Q4 <- brf %>%
  filter(!is.na(DIABAGE2), `_STATE` == 42, !(DIABAGE2 %in% c(98, 99))) %>%
  summarize(med_diab_age = median(DIABAGE2)) %>%
  as.data.frame()

Q4

med_diab_age
<dbl>
55


In [9]:
Q5 <- brf %>%
  filter((MENTHLTH %in% c(0, 88) | (MENTHLTH >= 1 & MENTHLTH <= 30))) %>%
  filter(!MARITAL %in% c(9)) %>%
  mutate(MENTHLTH = ifelse(MENTHLTH == 88, 0, MENTHLTH)) %>%
  mutate(MARITAL = factor(MARITAL)) %>%
  lm(MENTHLTH ~ MARITAL, data = .) %>%
  summary() 

Q5


Call:
lm(formula = MENTHLTH ~ MARITAL, data = .)

Residuals:
   Min     1Q Median     3Q    Max 
-7.027 -3.085 -2.497 -1.364 27.503 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  2.49739    0.01570  159.06   <2e-16 ***
MARITAL2     2.26240    0.03497   64.70   <2e-16 ***
MARITAL3     0.58725    0.03586   16.38   <2e-16 ***
MARITAL4     4.53000    0.08205   55.21   <2e-16 ***
MARITAL5     1.86635    0.03319   56.24   <2e-16 ***
MARITAL6     2.07783    0.06936   29.96   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 7.538 on 431112 degrees of freedom
Multiple R-squared:  0.01969,	Adjusted R-squared:  0.01968 
F-statistic:  1732 on 5 and 431112 DF,  p-value: < 2.2e-16


In [10]:
Q6 <- brf %>%
  filter((MENTHLTH %in% c(0, 88) | (MENTHLTH >= 1 & MENTHLTH <= 30))) %>%
  mutate(MENTHLTH = ifelse(MENTHLTH == 88, 0, MENTHLTH)) %>%
  filter(MARITAL != 9) %>%
  group_by(MARITAL) %>%
  summarise(mean_mental = mean(MENTHLTH, na.rm = TRUE)) %>%
  round(2) %>%
  as.data.frame()

Q6

MARITAL,mean_mental
<dbl>,<dbl>
1,2.50
2,4.76
3,3.08
4,7.03
5,4.36
6,4.58


In [11]:
Q7 <- brf %>%
  filter(HLTHPLN1 == 2) %>%
  filter((MENTHLTH %in% c(0, 88) | (MENTHLTH >= 1 & MENTHLTH <= 30))) %>%
  mutate(MENTHLTH = ifelse(MENTHLTH == 88, 0, MENTHLTH)) %>%
  group_by(CVDSTRK3) %>%
  summarise(
    mean_mental = mean(MENTHLTH, na.rm = TRUE),
    sd_mental = sd(MENTHLTH, na.rm = TRUE)
  ) %>%
  filter(CVDSTRK3 %in% c(1, 2)) %>%
  round(2) %>%
  as.data.frame()

Q7

CVDSTRK3,mean_mental,sd_mental
<dbl>,<dbl>,<dbl>
1,8.89,11.74
2,4.60,8.89


In [12]:
Q8 <- brf %>% 
  filter(!is.na(MARITAL), !is.na(EXEROFT1)) %>%
  filter(MARITAL != 9) %>%
  filter(between(EXEROFT1, 101, 199)) %>%
  mutate(MARITAL = factor(MARITAL)) %>%
  aov(EXEROFT1 ~ MARITAL, data = .) %>% 
  summary()

Q8

                Df  Sum Sq Mean Sq F value Pr(>F)    
MARITAL          5    7706  1541.1   177.9 <2e-16 ***
Residuals   206301 1787155     8.7                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [14]:
brfcopy <- data_frame(brf)

brfcopy_filtered <- brfcopy %>%
  filter(!is.na(EXRACT11), SEX == 1) %>%
  filter(!ALCDAY5 %in% c(201:299,777, 999)) %>%
  mutate(ALCDAY5 = ifelse(ALCDAY5 %in% 101:199, ALCDAY5 - 100, ALCDAY5)) %>%
  mutate(ALCDAY5 = ifelse(ALCDAY5 == 888, 0, ALCDAY5))

Q9 <- brfcopy_filtered %>%
  group_by(EXRACT11) %>%
  summarize(var_drinks = var(ALCDAY5, na.rm = TRUE)) %>%
  arrange(desc(var_drinks)) %>%
  head(6) %>%
  round(2)

Q9

EXRACT11,var_drinks
<dbl>,<dbl>
55,9.50
34,8.14
59,7.23
42,6.67
21,6.61
12,6.48
